<a href="https://colab.research.google.com/github/simran-sai/LLM-model/blob/main/LLM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.1 MB/s eta 0:00:00


In [2]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00


In [3]:
!pip install torch==1.13.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which

In [4]:
import pandas as pd
import torch
from transformers import TrainingArguments, Trainer, GPT2LMHeadModel, GPT2Tokenizer

In [5]:
df = pd.read_csv('Book2.csv')
df.head()

,Row No,Occasion,Relationship,Price Range,Interest,Style,Preference,Gift
0,1,Retirement,Friend,Low,Travel,Retro,Prefer unique designs,Vintage Suitcase
1,2,Farewell,Colleague,Medium,Art,Modern,Prefer practical gifts,Art Set
2,3,Valentine's Day,Partner,High,Music,Modern,Prefer convenience,Wireless Bluetooth Speaker
3,4,Job Promotion,Colleague,Low,Travel,Retro,Prefer aesthetic,Vintage Travel Journal
4,5,Retirement,Sibling,Medium,Sports,Modern,Likes aesthetic,Sports Watch


In [6]:
df.columns

Index(['Row No', 'Occasion', 'Relationship', 'Price Range', 'Interest',
       'Style', 'Preference', 'Gift'],
      dtype='object')

In [7]:
df.columns = df.columns.str.strip()


In [8]:
df['Preference']


0       Prefer unique designs
1      Prefer practical gifts
2          Prefer convenience
3            Prefer aesthetic
4             Likes aesthetic
                ...          
195        Prefer convenience
196     Prefer unique designs
197          Prefer aesthetic
198        Prefer convenience
199    Prefer practical gifts
Name: Preference, Length: 200, dtype: object

In [9]:
prompts = df.apply(lambda row: f"The occasion is {row['Occasion']}, the relationship is {row['Relationship']}, the range is {row['Price Range']},the interest is{['Interest']} ,the style is {row['Style']}, and the preference is {row['Preference']}. What is the appropriate gift?", axis=1)
answers = df['Gift']

In [10]:
prompts_and_answers = prompts + ' ' + answers
prompts_and_answers.to_csv('prompts_and_answers.txt', index=False)


In [11]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [12]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="prompts_and_answers.txt",
    block_size=128,
)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)


In [14]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-gifts", # The output directory
    overwrite_output_dir=True, # Overwrite the content of the output directory
    num_train_epochs=3, # The number of training epochs
    per_device_train_batch_size=1, # Batch size for training
    save_steps=10_000, # After # steps model is saved
    save_total_limit=2, # The total amount of checkpoints
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)


In [15]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.456800


TrainOutput(global_step=603, training_loss=0.4209645810609631, metrics={'train_runtime': 68.2362, 'train_samples_per_second': 8.837, 'train_steps_per_second': 8.837, 'total_flos': 14994692352000.0, 'train_loss': 0.4209645810609631, 'epoch': 3.0})

In [16]:
model.save_pretrained("./gpt2-gifts")


In [18]:
from transformers import pipeline

# Load the trained model.
model = GPT2LMHeadModel.from_pretrained("./gpt2-gifts")

# Use the pipeline for prediction
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
result = generator("The occasion is Retirement, the relationship is friend, the range is low, the interest is travel ,the style is retro, and the preference is Prefer unique designs. What is the appropriate gift?", max_length=60, temperature=0.7)

# Extract the generated text from the result and print it.
generated_text = result[0]['generated_text']
print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The occasion is Retirement, the relationship is friend, the range is low, the interest is travel ,the style is retro, and the preference is Prefer unique designs. What is the appropriate gift? Vintage Film Camera" is the appropriate gift. What is the appropriate gift? Film Camera" is the
